## 다중 인덱스, 프레임 병함

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

from datetime import date,datetime,timedelta
from dateutil.parser import parse

import json
print('numpy_version',np.__version__)
print('pandas_version',pd.__version__)

numpy_version 1.20.3
pandas_version 1.3.2


In [2]:
def seriesinfo(ary):
    print('data \n',ary)
    print('index:',type(ary.index),ary.index)
    print('value:',type(ary.values),ary.values)
def dfinfo(df):
    display(df)
    print('shape:',df.shape)
    print('size:',df.size)
    print('ndim:',df.ndim)
    print('index:',df.index)     # row index
    print('columns:',df.columns) # column inex 
    print('values:\n',df.values,type(df.values)) #ndarray 형식으로 나온다

### 다중인덱스

In [22]:
np.random.seed(100)
multi_df = pd.DataFrame(np.random.randn(5,4),columns=[['Group01','Group01','Group02','Group02'],
                                                      ['col01','col02','col03','col04']])
display(multi_df)
print(multi_df.columns)
print(multi_df.columns.names)

Group01             Group02          
      col01     col02     col03     col04
0 -1.749765  0.342680  1.153036 -0.252436
1  0.981321  0.514219  0.221180 -1.070043
2 -0.189496  0.255001 -0.458027  0.435163
3 -0.583595  0.816847  0.672721 -0.104411
4 -0.531280  1.029733 -0.438136 -1.118318

MultiIndex([('Group01', 'col01'),
            ('Group01', 'col02'),
            ('Group02', 'col03'),
            ('Group02', 'col04')],
           )
[None, None]


In [25]:
# 다중 index로 만들면 name이 아니라 names 속성이다
multi_df.columns.names=['Group_Id','Col_Id']
print(multi_df.columns.names)
print(multi_df.index.name) # 얘는 multi가 아니다


['Group_Id', 'Col_Id']
None


In [11]:
# 다르다
np.random.seed(100)
multi_df = pd.DataFrame(np.random.randn(5,4),columns=[('Group01','col01'),('Group02','col02'),('Group03','col03'),('Group04','col04')])
display(multi_df)

,"(Group01, col01)","(Group02, col02)","(Group03, col03)","(Group04, col04)"
0,-1.749765,0.342680,1.153036,-0.252436
1,0.981321,0.514219,0.221180,-1.070043
2,-0.189496,0.255001,-0.458027,0.435163
3,-0.583595,0.816847,0.672721,-0.104411
4,-0.531280,1.029733,-0.438136,-1.118318


In [48]:
# index columns 둘다 가능하다
np.random.seed(100)
multi_df = pd.DataFrame(np.random.randn(5,4),
                        columns=[['Group01','Group01','Group02','Group02'],
                                                      ['col01','col02','col01','col02']],
                        index = [['M','M','M','F','F'],
                                 [i for i in range(5)]])
multi_df.columns.names=['Group_Id','Col_Id']
multi_df.index.names=['Gender','Row_Id']
display(multi_df)

Group_Id        Group01             Group02          
Col_Id            col01     col02     col01     col02
Gender Row_Id                                        
M      0      -1.749765  0.342680  1.153036 -0.252436
       1       0.981321  0.514219  0.221180 -1.070043
       2      -0.189496  0.255001 -0.458027  0.435163
F      3      -0.583595  0.816847  0.672721 -0.104411
       4      -0.531280  1.029733 -0.438136 -1.118318

#### stack()
- 열이 행으로 들어온다

In [49]:
multi_df.stack()   

Group_Id               Group01   Group02
Gender Row_Id Col_Id                    
M      0      col01  -1.749765  1.153036
              col02   0.342680 -0.252436
       1      col01   0.981321  0.221180
              col02   0.514219 -1.070043
       2      col01  -0.189496 -0.458027
              col02   0.255001  0.435163
F      3      col01  -0.583595  0.672721
              col02   0.816847 -0.104411
       4      col01  -0.531280 -0.438136
              col02   1.029733 -1.118318

In [50]:
multi_df.stack('Group_Id') # multi_df.stack(0) 과 동일 

Col_Id                     col01     col02
Gender Row_Id Group_Id                    
M      0      Group01  -1.749765  0.342680
              Group02   1.153036 -0.252436
       1      Group01   0.981321  0.514219
              Group02   0.221180 -1.070043
       2      Group01  -0.189496  0.255001
              Group02  -0.458027  0.435163
F      3      Group01  -0.583595  0.816847
              Group02   0.672721 -0.104411
       4      Group01  -0.531280  1.029733
              Group02  -0.438136 -1.118318

#### unstack

In [51]:
multi_df.unstack()   

Group_Id   Group01                                                            \
Col_Id       col01                                           col02             
Row_Id           0         1         2         3        4        0         1   
Gender                                                                         
F              NaN       NaN       NaN -0.583595 -0.53128      NaN       NaN   
M        -1.749765  0.981321 -0.189496       NaN      NaN  0.34268  0.514219   

Group_Id                                 Group02                               \
Col_Id                                     col01                                
Row_Id           2         3         4         0        1         2         3   
Gender                                                                          
F              NaN  0.816847  1.029733       NaN      NaN       NaN  0.672721   
M         0.255001       NaN       NaN  1.153036  0.22118 -0.458027       NaN   

Group_Id                                                              
Col_Id                 col02                                          
Row_Id           4         0         1         2         3         4  
Gender                                                                
F        -0.438136       NaN       NaN       NaN -0.104411 -1.118318  
M              NaN -0.252436 -1.070043  0.435163       NaN       NaN

#### 다중 인덱스에서 인덱싱

In [52]:
multi_df[('Group01','col01')]

Gender  Row_Id
M       0        -1.749765
        1         0.981321
        2        -0.189496
F       3        -0.583595
        4        -0.531280
Name: (Group01, col01), dtype: float64

In [53]:
multi_df[[('Group01','col01')]]

Group_Id        Group01
Col_Id            col01
Gender Row_Id          
M      0      -1.749765
       1       0.981321
       2      -0.189496
F      3      -0.583595
       4      -0.531280

In [54]:
multi_df[[('Group01','col01'),('Group01','col02')]]

Group_Id        Group01          
Col_Id            col01     col02
Gender Row_Id                    
M      0      -1.749765  0.342680
       1       0.981321  0.514219
       2      -0.189496  0.255001
F      3      -0.583595  0.816847
       4      -0.531280  1.029733

In [59]:
print(multi_df[('Group02','col01')][0])
print(multi_df.loc[('M',0),('Group02','col01')])

1.153035802563644
1.153035802563644


In [ ]:
multi_df.loc[[('M','id_1')],:]

In [63]:
multi_df.loc[[('M',0)]]

Group_Id        Group01            Group02          
Col_Id            col01    col02     col01     col02
Gender Row_Id                                       
M      0      -1.749765  0.34268  1.153036 -0.252436

In [44]:
# Return type이 되게 신기하다
multi_df['Group01']

Col_Id            col01     col02
Gender Row_Id                    
M      0      -1.749765  0.342680
       1       0.981321  0.514219
       2      -0.189496  0.255001
F      3      -0.583595  0.816847
       4      -0.531280  1.029733

In [65]:
multi_df.loc[['M']]

Group_Id        Group01             Group02          
Col_Id            col01     col02     col01     col02
Gender Row_Id                                        
M      0      -1.749765  0.342680  1.153036 -0.252436
       1       0.981321  0.514219  0.221180 -1.070043
       2      -0.189496  0.255001 -0.458027  0.435163

#### 다중 인덱스 정렬
- level 속성을 이용해서 기준을 정의해야한다
- axis 이용해서 축 지정

In [68]:
np.random.seed(100)
multi_df = pd.DataFrame(np.random.randint(1,10,(5,4)),
                        columns=[['Group01','Group01','Group02','Group02'],
                                                      ['col01','col02','col01','col02']],
                        index = [['M','M','M','F','F'],
                                 [i for i in range(5)]])
multi_df.columns.names=['Group_Id','Col_Id']
multi_df.index.names=['Gender','Row_Id']
display(multi_df)

Group_Id      Group01       Group02      
Col_Id          col01 col02   col01 col02
Gender Row_Id                            
M      0            9     9       4     8
       1            8     1       5     3
       2            6     3       3     3
F      3            2     1       9     5
       4            1     7       3     5

In [73]:
multi_df.sort_index(level='Gender')

Group_Id      Group01       Group02      
Col_Id          col01 col02   col01 col02
Gender Row_Id                            
F      3            2     1       9     5
       4            1     7       3     5
M      0            9     9       4     8
       1            8     1       5     3
       2            6     3       3     3

In [75]:
multi_df.sort_index(level=0,axis=0)

Group_Id      Group01       Group02      
Col_Id          col01 col02   col01 col02
Gender Row_Id                            
F      3            2     1       9     5
       4            1     7       3     5
M      0            9     9       4     8
       1            8     1       5     3
       2            6     3       3     3

In [74]:
multi_df.sort_index(level='Row_Id')

Group_Id      Group01       Group02      
Col_Id          col01 col02   col01 col02
Gender Row_Id                            
M      0            9     9       4     8
       1            8     1       5     3
       2            6     3       3     3
F      3            2     1       9     5
       4            1     7       3     5

In [77]:
multi_df.sort_index(level=1,axis=0)

Group_Id      Group01       Group02      
Col_Id          col01 col02   col01 col02
Gender Row_Id                            
M      0            9     9       4     8
       1            8     1       5     3
       2            6     3       3     3
F      3            2     1       9     5
       4            1     7       3     5

### 병합
- merge : 공통의 열을 기준으로 병합한다 => inner join
- concat : 연결
- join : 인덱스를 기준으로 병합

In [78]:
data01 = {
    '학번' : [1,2,3,4],
    '이름' : ['lee','park','kim','choi'],
    '학년' : [2,4,1,3]
}

data02 = {
    '학번' : [3,2,4,5],
    '학과' : ['CS','AI','AI','BIZ'],
    '학점' : [2.1,4.1,1.4,3.3]
}

In [79]:
student_df = pd.DataFrame(data01)
major_df = pd.DataFrame(data02)
display(student_df)
display(major_df)

,학번,이름,학년
0,1,lee,2
1,2,park,4
2,3,kim,1
3,4,choi,3


,학번,학과,학점
0,3,CS,2.1
1,2,AI,4.1
2,4,AI,1.4
3,5,BIZ,3.3


In [82]:
pd.merge(student_df,major_df,how='inner')

,학번,이름,학년,학과,학점
0,2,park,4,AI,4.1
1,3,kim,1,CS,2.1
2,4,choi,3,AI,1.4


In [83]:
pd.merge(student_df,major_df,how='outer')

,학번,이름,학년,학과,학점
0,1,lee,2.0,NaN,NaN
1,2,park,4.0,AI,4.1
2,3,kim,1.0,CS,2.1
3,4,choi,3.0,AI,1.4
4,5,NaN,NaN,BIZ,3.3


In [84]:
pd.merge(student_df,major_df,how='left')

,학번,이름,학년,학과,학점
0,1,lee,2,NaN,NaN
1,2,park,4,AI,4.1
2,3,kim,1,CS,2.1
3,4,choi,3,AI,1.4


In [85]:
pd.merge(student_df,major_df,how='right')

,학번,이름,학년,학과,학점
0,3,kim,1.0,CS,2.1
1,2,park,4.0,AI,4.1
2,4,choi,3.0,AI,1.4
3,5,NaN,NaN,BIZ,3.3


#### 열이 다 다르면?

In [88]:
data01 = {
    '학번' : [1,2,3,4],
    '이름' : ['lee','park','kim','choi'],
    '학년' : [2,4,1,3]
}

data02 = {
    '학생번호' : [3,2,4,5],
    '학과' : ['CS','AI','AI','BIZ'],
    '학점' : [2.1,4.1,1.4,3.3]
}
student_df = pd.DataFrame(data01)
major_df = pd.DataFrame(data02)
display(student_df)
display(major_df)

,학번,이름,학년
0,1,lee,2
1,2,park,4
2,3,kim,1
3,4,choi,3


,학생번호,학과,학점
0,3,CS,2.1
1,2,AI,4.1
2,4,AI,1.4
3,5,BIZ,3.3


In [89]:
pd.merge(student_df,major_df,how='inner',left_on='학번',right_on='학생번호')

,학번,이름,학년,학생번호,학과,학점
0,2,park,4,2,AI,4.1
1,3,kim,1,3,CS,2.1
2,4,choi,3,4,AI,1.4


#### 중복이 되면? => 가능한 모든 조합

In [90]:
import seaborn as sns
datasets = sns.load_dataset('iris')
print('type - ')
print( type(datasets))

type - 
<class 'pandas.core.frame.DataFrame'>


In [91]:
datasets.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [95]:
data01 = {
    'species' : ['sectosa','virginica','virginica','versicolor'],
    'sepal_length' : [ 5.1,4.9,4.7,4.6]
}

data02 = {
    'species' : ['sectosa','sectosa','virginica','virginica'],
    'sepal_width' : [ 3.5,3.0,3.2,3.1]
}

df01 = pd.DataFrame(data01)
df02 = pd.DataFrame(data02)

In [97]:
# 가능한 조합을 다 가져온다 
pd.merge(df01,df02)

,species,sepal_length,sepal_width
0,sectosa,5.1,3.5
1,sectosa,5.1,3.0
2,virginica,4.9,3.2
3,virginica,4.9,3.1
4,virginica,4.7,3.2
5,virginica,4.7,3.1


#### 중복되는 키가 여러개 있을경우?

In [104]:
data01 = {
    'species' : ['sectosa','virginica','virginica','versicolor'],
    'sepal_length' : [ 5.1,4.9,4.7,4.6],
}

data02 = {
    'species' : ['sectosa','sectosa','virginica','virginica'],
    'sepal_length' : [ 3.5,3.0,3.2,3.1]
}

df01 = pd.DataFrame(data01)
df02 = pd.DataFrame(data02)

In [106]:
pd.merge(df01,df02)

,species,sepal_length


In [107]:
pd.merge(df01,df02,on='species')

,species,sepal_length_x,sepal_length_y
0,sectosa,5.1,3.5
1,sectosa,5.1,3.0
2,virginica,4.9,3.2
3,virginica,4.9,3.1
4,virginica,4.7,3.2
5,virginica,4.7,3.1


#### 컬럼 인덱스가 아닌 인덱스를 기준으로 병합 한다면?

In [108]:
pop01_df = pd.DataFrame({
    'city' : ['seoul','seoul','seoul','daejun','daejun'],
    'year' : [2010,2011,2020,2018,2015],
    'pop': [1234135,23413413,2313423,134323,42341]
})
pop01_df

,city,year,pop
0,seoul,2010,1234135
1,seoul,2011,23413413
2,seoul,2020,2313423
3,daejun,2018,134323
4,daejun,2015,42341


In [115]:
pop02_df = pd.DataFrame(np.arange(10).reshape(5,2),
                       columns=['col01','col02'],
                       index=[
                           ['seoul','seoul','daejun','daejun','daejun'],
                           [2010,2050,2011,2012,2015]
                       ])

pop02_df.index.names=('city','year')
pop02_df

col01  col02
city   year              
seoul  2010      0      1
       2050      2      3
daejun 2011      4      5
       2012      6      7
       2015      8      9

In [118]:
pd.merge(pop01_df,pop02_df,right_index=True,left_on=['city','year'])

,city,year,pop,col01,col02
0,seoul,2010,1234135,0,1
4,daejun,2015,42341,8,9


In [132]:
pop02_df.stack()

city    year       
seoul   2010  col01    0
              col02    1
        2050  col01    2
              col02    3
daejun  2011  col01    4
              col02    5
        2012  col01    6
              col02    7
        2015  col01    8
              col02    9
dtype: int64

In [133]:
pd.merge(pop01_df,pop02_df.stack(),right_index=True,left_on=['city'])

ValueError: Cannot merge a Series without a name

 <hr style="border:2px solid blue"> </hr>

In [135]:
data1 = {
    '학번' : [1,2,3,4] , 
    '이름' : ['려명' , '승윤' , '태형' , '지엽'] , 
    '학년' : [2,4,1,3]
}
data2 = {
    '학생번호' : [1,2,4,5] , 
    '학과' : ['CS' , 'AI' , 'AI' , 'CS'] , 
    '학점' : [2.4 , 4.5 , 1.7 , 3.9]
}
stu_df   = pd.DataFrame(data1)
major_df = pd.DataFrame(data2)
display(stu_df)
display(major_df)

,학번,이름,학년
0,1,려명,2
1,2,승윤,4
2,3,태형,1
3,4,지엽,3


,학생번호,학과,학점
0,1,CS,2.4
1,2,AI,4.5
2,4,AI,1.7
3,5,CS,3.9


In [136]:
pd.merge(stu_df,major_df,right_index=True,left_index=True)

,학번,이름,학년,학생번호,학과,학점
0,1,려명,2,1,CS,2.4
1,2,승윤,4,2,AI,4.5
2,3,태형,1,4,AI,1.7
3,4,지엽,3,5,CS,3.9


### join

In [137]:
stu_df.join(major_df)

,학번,이름,학년,학생번호,학과,학점
0,1,려명,2,1,CS,2.4
1,2,승윤,4,2,AI,4.5
2,3,태형,1,4,AI,1.7
3,4,지엽,3,5,CS,3.9


In [138]:
stu_df.set_index('학번',inplace=True)
major_df.set_index('학생번호',inplace=True)
display(stu_df)
display(major_df)

,이름,학년
학번,,
1,려명,2
2,승윤,4
3,태형,1
4,지엽,3


,학과,학점
학생번호,,
1,CS,2.4
2,AI,4.5
4,AI,1.7
5,CS,3.9


In [141]:
stu_df.join(major_df)

,이름,학년,학과,학점
학번,,,,
1,려명,2,CS,2.4
2,승윤,4,AI,4.5
3,태형,1,NaN,NaN
4,지엽,3,AI,1.7


In [140]:
stu_df.join(major_df,how='inner')

,이름,학년,학과,학점
1,려명,2,CS,2.4
2,승윤,4,AI,4.5
4,지엽,3,AI,1.7


### concat

### Group

In [144]:
import seaborn as sns
iris_df = sns.load_dataset('iris')
print('type - ')
print( type(iris_df))
iris_df

type - 
<class 'pandas.core.frame.DataFrame'>


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [159]:
type(iris_df.groupby('species'))

pandas.core.groupby.generic.DataFrameGroupBy

In [161]:
# iris 종별 가장 큰 값과 작은 값으 비율을 구한다면 ?
iris_df.groupby('species').max()/iris_df.groupby('species').min()

# agg이용
iris_df.groupby('species').agg(lambda x:x.max()/x.min())

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


In [150]:
# iris 종별 가장 큰 pental_length 3개를 구한다면?
iris_df.sort_values('petal_length',ascending=False).groupby('species').head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
118,7.7,2.6,6.9,2.3,virginica
122,7.7,2.8,6.7,2.0,virginica
117,7.7,3.8,6.7,2.2,virginica
83,6.0,2.7,5.1,1.6,versicolor
77,6.7,3.0,5.0,1.7,versicolor
52,6.9,3.1,4.9,1.5,versicolor
24,4.8,3.4,1.9,0.2,setosa
44,5.1,3.8,1.9,0.4,setosa
5,5.4,3.9,1.7,0.4,setosa
